In [ ]:
import numpy as np
import matplotlib.pyplot as plt

T=911
sigma=0.412
c=np.sqrt(T/sigma)

r_1=0.175  
r_2 = 0.247  # Radios de los tambores circulares
dr= 0.0035 # Delta de r

nth=60 # Número de iteraciones en theta
dth=2*np.pi/nth # Delta de theta
th = np.linspace(0, 2 * np.pi, nth, endpoint=False)  # Barrido de la coordenada angular

dt = 0.00001  # Delta t
t = 0.1 # Tiempo total
nt = int(t / dt) # Número de pasos temporales

def mdf_circ(R):
   nr=int(R/dr)
   r = np.linspace(dr, R, nr + 1)  # Barrido de la coordenada radial
   theta = np.linspace(0, 2 * np.pi, nth) # Barrido de la coordenada angular
   u = np.zeros((nr + 1, nth, 3))  # 3 arrays de (r, theta, t), para n-1, n y  n+1
   u[nr//2, 0, 1] = 10 # Condición inicial, golpe de la baqueta
   
   for n in range(1, nt):
      for i in range(1, nr):
         for j in range(0, nth):
            jp = (j + 1) % nth  # Condición periódica en theta (j+1)
            jm = (j - 1) % nth  # Condición periódica en theta (j-1)

             # Derivada segunda de u con respecto a r dos veces
            d2u_dr2 = (u[i + 1, j, 1] - 2 * u[i, j, 1] + u[i - 1, j, 1]) / dr**2
             # Derivada de u con respecto a r
            du_dr = (u[i + 1, j, 1] - u[i - 1, j, 1]) / (2 * dr * r[i])
             # Derivada segunda de u con respecto a theta dos veces
            d2u_dtheta2 = (u[i, jp, 1] - 2 * u[i, j, 1] + u[i, jm, 1]) / (r[i]**2 * dth**2)

             # u(n+1)
            u[i, j, 2] = (2 * u[i, j, 1] - u[i, j, 0]
                           + (c * dt)**2 * (d2u_dr2 + du_dr + d2u_dtheta2))

      u[0, :, 2] = 0  # Condiciones de contorno de Dirichlet en r=0
      u[-1, :, 2] = 0  # Condiciones de contorno de Dirichlet en r=R

      u[:, :, 0] = u[:, :, 1]  # n-1 <- n
      u[:, :, 1] = u[:, :, 2]  # n <- n+1
   R, Theta = np.meshgrid(r, theta)
   X = R * np.cos(Theta)
   Y = R * np.sin(Theta)

   plt.figure(figsize=(8, 6))
   plt.pcolormesh(X, Y, u[:, :, 1].T, cmap="rainbow", shading="auto")
   plt.colorbar(label="Amplitud")
   plt.title("Membrana circular: Método de diferencias finitas")
   plt.xlabel("x (m)")
   plt.ylabel("y (m)")
   plt.axis("equal")
   plt.show()

mdf_circ(r_1)
mdf_circ(r_2)